# Running hyperparameter optimization on Chemprop model using RayTune

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chemprop/chemprop/blob/main/examples/hpopting.ipynb)

In [1]:
# Install chemprop from GitHub if running in Google Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    try:
        import chemprop
    except ImportError:
        !git clone https://github.com/chemprop/chemprop.git
        %cd chemprop
        !pip install ".[hpopt]"
        %cd examples

## Import packages

In [2]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

In [3]:
chemprop_dir = Path.cwd().parent
input_path = chemprop_dir / "tests" / "data" / "regression" / "mol" / "mol.csv" # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['lipo'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

## Load data

In [4]:
df_input = pd.read_csv(input_path)
df_input

,smiles,lipo
0,Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14,3.54
1,COc1cc(OC)c(cc1NC(=O)CSCC(=O)O)S(=O)(=O)N2C(C)...,-1.18
2,COC(=O)[C@@H](N1CCc2sccc2C1)c3ccccc3Cl,3.69
3,OC[C@H](O)CN1C(=O)C(Cc2ccccc12)NC(=O)c3cc4cc(C...,3.37
4,Cc1cccc(C[C@H](NC(=O)c2cc(nn2C)C(C)(C)C)C(=O)N...,3.10
...,...,...
95,CC(C)N(CCCNC(=O)Nc1ccc(cc1)C(C)(C)C)C[C@H]2O[C...,2.20
96,CCN(CC)CCCCNc1ncc2CN(C(=O)N(Cc3cccc(NC(=O)C=C)...,2.04
97,CCSc1c(Cc2ccccc2C(F)(F)F)sc3N(CC(C)C)C(=O)N(C)...,4.49
98,COc1ccc(Cc2c(N)n[nH]c2N)cc1,0.20


In [5]:
smis = df_input.loc[:, smiles_column].values
ys = df_input.loc[:, target_columns].values

## Make data points, splits, and datasets

In [6]:
all_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(smis, ys)]

In [7]:
mols = [d.mol for d in all_data]  # RDkit Mol objects are use for structure based splits
train_indices, val_indices, test_indices = data.make_split_indices(mols, "random", (0.8, 0.1, 0.1))
train_data, val_data, test_data = data.split_data_by_indices(
    all_data, train_indices, val_indices, test_indices
)

The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)


In [8]:
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

train_dset = data.MoleculeDataset(train_data[0], featurizer)
scaler = train_dset.normalize_targets()

val_dset = data.MoleculeDataset(val_data[0], featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data[0], featurizer)


# Define helper function to train the model

In [9]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)


## Define parameter search space

In [10]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [11]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

def train_func(config):
    trainer = TorchTrainer(
        lambda cfg: train_model(
            cfg, train_dset, val_dset, num_workers, scaler
        ),
        scaling_config=scaling_config,
        run_config=run_config,
        train_loop_config=config,
    )
    result = trainer.fit()
    tune.report(metrics=result.metrics, checkpoint=result.checkpoint)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch() 

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths
    
)

tuner = tune.Tuner(
    train_func,
    param_space=search_space,
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()


(TrainController pid=4881) Attempting to start training worker group of size 1 with the following resources: [{'CPU': 1}] * 1
(RayTrainWorker pid=5060) Setting up process group for: env:// [rank=0, world_size=1]
(pid=gcs_server) [2025-12-27 21:31:15,719 E 3949 3949] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(TrainController pid=5197) Attempting to start training worker group of size 1 with the following resources: [{'CPU': 1}] * 1
(raylet) [2025-12-27 21:31:16,995 E 4073 4073] (raylet) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
(bundle_reservation_check_func pid=4142) [2025-12-27 21:31:18,823 E 4142 4309] core_worker_process.cc

(RayTrainWorker pid=5060) [Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


(TrainController pid=4881) Started training worker group of size 1: 
(TrainController pid=4881) - (ip=172.31.231.162, pid=5060) world_rank=0, local_rank=0, node_rank=0
(train_func pid=4775) [2025-12-27 21:31:22,286 E 4775 4821] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14 [repeated 7x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(RayTrainWorker pid=5060) INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=5060) 💡 Tip: For seamless cloud uploads and version

(RayTrainWorker pid=5060) ┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
(RayTrainWorker pid=5060) ┃   ┃ Name            ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
(RayTrainWorker pid=5060) ┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
(RayTrainWorker pid=5060) │ 0 │ message_passing │ BondMessagePassing │  579 K │ train │     0 │
(RayTrainWorker pid=5060) │ 1 │ agg             │ MeanAggregation    │      0 │ train │     0 │
(RayTrainWorker pid=5060) │ 2 │ bn              │ BatchNorm1d        │  1.0 K │ train │     0 │
(RayTrainWorker pid=5060) │ 3 │ predictor       │ RegressionFFN      │  5.0 M │ train │     0 │
(RayTrainWorker pid=5060) │ 4 │ X_d_transform   │ Identity           │      0 │ train │     0 │
(RayTrainWorker pid=5060) │ 5 │ metrics         │ ModuleList         │      0 │ train │     0 │
(RayTrainWorker pid=5060) └───┴─────────────────┴────────────────────┴────────┴───────┴───────┘
(RayTrainWorker pid=5060) Trainable para

(RayTrainWorker pid=5261) [W1227 21:31:30.086113743 socket.cpp:209] [c10d] The hostname of the client socket cannot be retrieved. err=-3


...

`Trainer.fit` stopped: `max_epochs=20` reached.
(RayTrainWorker pid=5060)                                                               train_loss_step:  
(RayTrainWorker pid=5060)                                                               0.727             
(RayTrainWorker pid=5060)                                                               train_loss_epoch: 
(RayTrainWorker pid=5060)                                                               train_loss_step:  
(RayTrainWorker pid=5060)                                                               0.707             
(RayTrainWorker pid=5060)                                                               train_loss_epoch: 
(RayTrainWorker pid=5060)                                                               0.110             
(RayTrainWorker pid=5060) 
Epoch 19/19 ━━━━━━━━━━━━━━━━━━ 2/2 0:00:00 • 0:00:00 7.29it/s v_num: 0.000       [repeated 29x across cluster]
Epoch 19/19 ━━━━━━━━━━━━━━━━━━ 2/2 0:00:00 • 0:00:00 7.29it/s v_n

(train_func pid=4775) /home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(train_func pid=4775)   _log_deprecation_warning(
(train_func pid=4775) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/knathan/ray_results/train_func_2025-12-27_21-30-50/36f1861c/checkpoint_000000)


(RayTrainWorker pid=5261)                                                               train_loss_step:  
(RayTrainWorker pid=5261)                                                               0.812             
(RayTrainWorker pid=5261)                                                               train_loss_epoch: 
(RayTrainWorker pid=5261)                                                               train_loss_step:  
(RayTrainWorker pid=5261)                                                               0.812             
(RayTrainWorker pid=5261)                                                               train_loss_epoch: 
(RayTrainWorker pid=5261)                                                               0.090             
(RayTrainWorker pid=5261)                                                               train_loss_step:  
(RayTrainWorker pid=5261)                                                               0.812             
(RayTrainWorker pid=5261)            

(RayTrainWorker pid=5261) INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
(train_func pid=4880) /home/knathan/anaconda3/envs/chemprop/lib/python3.11/site-packages/ray/tune/trainable/trainable_fn_utils.py:41: RayDeprecationWarning: The `Checkpoint` class should be imported from `ray.tune` when passing it to `ray.tune.report` in a Tune function. Please update your imports. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(train_func pid=4880)   _log_deprecation_warning(
2025-12-27 21:32:19,738	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/knathan/ray_results/train_func_2025-12-27_21-30-50' in 0.0081s.
2025-12-27 21:32:19,748	INFO tune.py:1041 -- Total run time: 89.24 seconds (89.14 seconds for the tuning loop).


(train_func pid=4880) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/knathan/ray_results/train_func_2025-12-27_21-30-50/f4fb5e5c/checkpoint_000000)


## Hyperparameter optimization results

In [12]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.10997936874628067, 'train_loss_step': 0.20683002471923828, 'val/rmse': 0.8407691717147827, 'val/mae': 0.6798940896987915, 'val_loss': 0.7068928480148315, 'train_loss_epoch': 0.10997936874628067, 'epoch': 19, 'step': 40},
    path='/home/knathan/ray_results/train_func_2025-12-27_21-30-50/36f1861c',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/home/knathan/ray_results/train_func_2025-12-27_21-30-50/36f1861c/checkpoint_000000)
  ),
  Result(
    metrics={'train_loss': 0.07758364081382751, 'train_loss_step': 0.12734931707382202, 'val/rmse': 0.863564670085907, 'val/mae': 0.7058178186416626, 'val_loss': 0.745743989944458, 'train_loss_epoch': 0.07758364081382751, 'epoch': 19, 'step': 40},
    path='/home/knathan/ray_results/train_func_2025-12-27_21-30-50/f4fb5e5c',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/home/knathan/ray_results/train_func_2025-12-27_21-30-50/f4fb5e5c/checkpoint_

In [13]:
# results of all trials
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/depth,config/ffn_hidden_dim,config/ffn_num_layers,config/message_hidden_dim,logdir
0,0.109979,0.206830,0.840769,0.679894,0.706893,0.109979,19,40,1766896329,checkpoint_000000,...,4775,Knathan-Laptop,172.31.231.162,67.695003,1,2,2000,2,500,36f1861c
1,0.077584,0.127349,0.863565,0.705818,0.745744,0.077584,19,40,1766896339,checkpoint_000000,...,4880,Knathan-Laptop,172.31.231.162,66.469962,1,2,2200,2,400,f4fb5e5c


In [14]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config

{'depth': 2,
 'ffn_hidden_dim': 2000,
 'ffn_num_layers': 2,
 'message_hidden_dim': 500}

In [15]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /home/knathan/ray_results/train_func_2025-12-27_21-30-50/36f1861c/checkpoint_000000/checkpoint.ckpt


In [16]:
ray.shutdown()